# Situp Counter - Camera & Video Analysis (Windows CPU Optimized)

This notebook loads the trained situp counter model and analyzes situps from:
- 🎥 **Live Camera**: Real-time webcam counting (default 30 seconds)
- 📹 **Video File**: Analyze pre-recorded situp videos
- 🚀 **CPU Optimized**: Maximum performance on Windows (GPU doesn't help!)

## ⚠️ IMPORTANT: Windows + GPU Reality
**GPU acceleration does NOT work for MediaPipe on Windows** - even with CUDA installed!
- This is a MediaPipe limitation (not your hardware)
- The code is already optimized for maximum CPU performance
- Expected speed: 20-30 FPS on modern CPUs (totally fine!)
- See `WINDOWS_GPU_REALITY.md` for full explanation

## Quick Start
1. Run cells 1-15 to set up the environment
2. **Run cell 16** for Windows CPU optimization 🚀
3. Choose your input method:
   - **Cell 19**: Manual selection (uncomment the option you want)
   - **Cell 21**: Interactive menu (easier - just run it)
4. Use `use_gpu=False` (already set) - it's faster on Windows!

## 1. Install Required Libraries

In [9]:
!pip install mediapipe opencv-python numpy scikit-learn joblib cuda-python


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## 2. Import Libraries

In [10]:
import cv2
import mediapipe as mp
import numpy as np
import time
import joblib
import os

print("✓ Libraries imported successfully!")
print(f"MediaPipe version: {mp.__version__}")
print(f"OpenCV version: {cv2.__version__}")

✓ Libraries imported successfully!
MediaPipe version: 0.10.21
OpenCV version: 4.11.0


## 3. Load the Trained Model

In [11]:
# Load the saved model
model_path = "../Models/situp_counter_model.pkl"

if os.path.exists(model_path):
    model = joblib.load(model_path)
    print(f"✓ Model loaded successfully from: {model_path}")
    print(f"Model type: {type(model).__name__}")
    print(f"Number of features: {model.n_features_in_}")
else:
    print(f"❌ Model not found at: {model_path}")
    print("Please run the training notebook first!")

✓ Model loaded successfully from: ../Models/situp_counter_model.pkl
Model type: RandomForestClassifier
Number of features: 4


## 4. Define Helper Functions

In [12]:
def calculate_angle(a, b, c):
    """
    Calculate angle between three points
    Args:
        a, b, c: Landmark points (x, y coordinates)
    Returns:
        angle: Angle in degrees
    """
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
        
    return angle

def extract_pose_features(landmarks):
    """
    Extract relevant features from pose landmarks for situp detection
    """
    # Get key landmarks
    left_shoulder = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value].x,
                     landmarks[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value].y]
    left_hip = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_HIP.value].x,
                landmarks[mp.solutions.pose.PoseLandmark.LEFT_HIP.value].y]
    left_knee = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_KNEE.value].x,
                 landmarks[mp.solutions.pose.PoseLandmark.LEFT_KNEE.value].y]
    
    right_shoulder = [landmarks[mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                      landmarks[mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER.value].y]
    right_hip = [landmarks[mp.solutions.pose.PoseLandmark.RIGHT_HIP.value].x,
                 landmarks[mp.solutions.pose.PoseLandmark.RIGHT_HIP.value].y]
    right_knee = [landmarks[mp.solutions.pose.PoseLandmark.RIGHT_KNEE.value].x,
                  landmarks[mp.solutions.pose.PoseLandmark.RIGHT_KNEE.value].y]
    
    # Calculate angles
    left_hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)
    right_hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)
    
    # Average hip angle
    avg_hip_angle = (left_hip_angle + right_hip_angle) / 2
    
    # Torso vertical position
    torso_position = (left_shoulder[1] + right_shoulder[1]) / 2
    
    return [avg_hip_angle, torso_position, left_hip[1], right_hip[1]]

print("✓ Helper functions defined!")

✓ Helper functions defined!


## 5. Situp Counter Class

In [13]:
class SitupCounter:
    def __init__(self, angle_threshold_down=60, angle_threshold_up=100):
        """
        Initialize situp counter
        Args:
            angle_threshold_down: Hip angle threshold for down position
            angle_threshold_up: Hip angle threshold for up position
        """
        self.counter = 0
        self.stage = None  # 'down' or 'up'
        self.angle_threshold_down = angle_threshold_down
        self.angle_threshold_up = angle_threshold_up
        
    def update(self, hip_angle):
        """
        Update counter based on hip angle
        """
        # Down position (lying down)
        if hip_angle > self.angle_threshold_up:
            self.stage = "down"
        
        # Up position (sitting up)
        if hip_angle < self.angle_threshold_down and self.stage == "down":
            self.stage = "up"
            self.counter += 1
            
        return self.counter, self.stage
    
    def reset(self):
        """Reset counter"""
        self.counter = 0
        self.stage = None

print("✓ SitupCounter class ready!")

✓ SitupCounter class ready!


## 6. Real-time Situp Counter with Camera

**Instructions:**
- Position yourself so your full body is visible in the webcam
- Lie down with knees bent for the starting position
- The system will count your situps for 30 seconds
- Press 'q' to exit early

## 6A. GPU Support on Windows ⚠️

**IMPORTANT: MediaPipe GPU support is LIMITED on Windows!**

Even with CUDA installed, MediaPipe does **NOT** significantly accelerate on Windows. This is a MediaPipe limitation:
- ✅ **Linux/Mac**: Full GPU support with CUDA
- ⚠️ **Windows**: Minimal/no GPU acceleration (even with CUDA)

**For Windows users:**
- Use `use_gpu=False` (CPU mode) - it's actually the same speed!
- The code below is already optimized for CPU performance
- Expect ~20-30 FPS processing speed on modern CPUs

**Run Cell 7 below to verify your GPU setup** (it will confirm Windows limitations).

## 6B. Verify GPU/CUDA Setup

Run this cell to verify your GPU and CUDA are properly detected:

In [14]:
# Verify GPU and CUDA installation
import subprocess

print("🔍 GPU & CUDA VERIFICATION")
print("="*60)

# Check NVIDIA GPU
try:
    result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
    if result.returncode == 0:
        print("✅ NVIDIA GPU detected!")
        # Extract GPU name
        lines = result.stdout.split('\n')
        for line in lines:
            if 'NVIDIA' in line or 'GeForce' in line or 'RTX' in line:
                print(f"   {line.strip()}")
                break
    else:
        print("❌ nvidia-smi failed - GPU may not be available")
except FileNotFoundError:
    print("❌ nvidia-smi not found - NVIDIA drivers may not be installed")

print()

# Check PyTorch CUDA
try:
    import torch
    print("✅ PyTorch installed")
    print(f"   Version: {torch.__version__}")
    print(f"   CUDA Available: {torch.cuda.is_available()}")
    
    if torch.cuda.is_available():
        print(f"   CUDA Version: {torch.version.cuda}")
        print(f"   GPU Device: {torch.cuda.get_device_name(0)}")
        print(f"   GPU Count: {torch.cuda.device_count()}")
        print(f"   Current Device: {torch.cuda.current_device()}")
        
        # Test GPU
        try:
            x = torch.rand(5, 3).cuda()
            print("   ✅ GPU test successful!")
        except:
            print("   ⚠️  GPU test failed")
    else:
        print("   ⚠️  CUDA not available in PyTorch")
        print("   Install: pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118")
        
except ImportError:
    print("⚠️  PyTorch not installed")
    print("   Install: pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118")

print()

# Check TensorFlow (used by MediaPipe)
try:
    import tensorflow as tf
    print("✅ TensorFlow installed")
    print(f"   Version: {tf.__version__}")
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        print(f"   GPU Devices: {len(gpus)}")
        for gpu in gpus:
            print(f"   - {gpu.name}")
        print("   ✅ TensorFlow can use GPU!")
    else:
        print("   ⚠️  No GPU devices found by TensorFlow")
        print("   Install GPU version: pip install tensorflow-gpu")
except ImportError:
    print("ℹ️  TensorFlow not installed (MediaPipe has its own runtime)")

print()
print("="*60)
print("💡 SUMMARY:")
print("   - If CUDA is available, MediaPipe will use it automatically")
print("   - GPU support improves processing speed")
print("   - You should see faster processing compared to CPU-only")
print("="*60)

🔍 GPU & CUDA VERIFICATION
✅ NVIDIA GPU detected!
   | NVIDIA-SMI 556.29                 Driver Version: 556.29         CUDA Version: 12.5     |

⚠️  PyTorch not installed
   Install: pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118

ℹ️  TensorFlow not installed (MediaPipe has its own runtime)

💡 SUMMARY:
   - If CUDA is available, MediaPipe will use it automatically
   - GPU support improves processing speed
   - You should see faster processing compared to CPU-only
✅ NVIDIA GPU detected!
   | NVIDIA-SMI 556.29                 Driver Version: 556.29         CUDA Version: 12.5     |

⚠️  PyTorch not installed
   Install: pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118

ℹ️  TensorFlow not installed (MediaPipe has its own runtime)

💡 SUMMARY:
   - If CUDA is available, MediaPipe will use it automatically
   - GPU support improves processing speed
   - You should see faster processing compared to CPU-only


In [15]:
# Windows Performance Optimization
import os
import multiprocessing

print("🚀 WINDOWS PERFORMANCE OPTIMIZATION")
print("="*60)

# Check CPU cores
cpu_cores = multiprocessing.cpu_count()
print(f"✅ CPU Cores Available: {cpu_cores}")

# Set CPU optimization flags
os.environ['OMP_NUM_THREADS'] = str(cpu_cores)
os.environ['MKL_NUM_THREADS'] = str(cpu_cores)

print(f"✅ Multi-threading enabled: Using {cpu_cores} cores")
print()
print("💡 PERFORMANCE TIPS FOR WINDOWS:")
print("   1. Use model_complexity=0 for fastest processing (lower accuracy)")
print("   2. Use model_complexity=1 for balanced speed/accuracy (recommended)")
print("   3. Close other heavy applications to free CPU resources")
print("   4. Expected speed: 20-30 FPS on modern CPUs")
print()
print("⚠️  GPU acceleration does NOT work on Windows for MediaPipe!")
print("   Even with CUDA, MediaPipe uses CPU on Windows.")
print("   This is a MediaPipe limitation, not your hardware.")
print("="*60)

🚀 WINDOWS PERFORMANCE OPTIMIZATION
✅ CPU Cores Available: 16
✅ Multi-threading enabled: Using 16 cores

💡 PERFORMANCE TIPS FOR WINDOWS:
   1. Use model_complexity=0 for fastest processing (lower accuracy)
   2. Use model_complexity=1 for balanced speed/accuracy (recommended)
   3. Close other heavy applications to free CPU resources
   4. Expected speed: 20-30 FPS on modern CPUs

⚠️  GPU acceleration does NOT work on Windows for MediaPipe!
   Even with CUDA, MediaPipe uses CPU on Windows.
   This is a MediaPipe limitation, not your hardware.


## 6C. Windows Performance Optimization 🚀

Since GPU doesn't help on Windows, here are CPU optimizations that WILL improve performance:

## 7. Main Situp Counter Function (CPU Optimized for Windows)

This is the main function that processes video/camera and counts situps.
**Optimized for Windows CPU performance** (GPU doesn't help on Windows):

In [16]:
def run_situp_counter(source='camera', video_path=None, duration=30, use_gpu=False):
    """
    Run situp counter with webcam or video file
    Args:
        source: 'camera' or 'video' - input source type
        video_path: Path to video file (required if source='video')
        duration: Time duration in seconds (default 30, only for camera mode)
        use_gpu: Try to use GPU acceleration (default False, limited support on Windows)
    """
    # Initialize MediaPipe Pose
    mp_pose = mp.solutions.pose
    mp_drawing = mp.solutions.drawing_utils
    
    # GPU info
    if use_gpu:
        print("⚡ GPU acceleration requested (limited support on Windows)")
    else:
        print("🖥️  Using CPU mode (recommended for Windows)")
    
    # Initialize situp counter
    situp_counter = SitupCounter()
    
    # Setup video capture based on source
    if source.lower() == 'camera':
        cap = cv2.VideoCapture(0)
        use_duration = True
        print(f"🎥 Starting situp counter with WEBCAM for {duration} seconds...")
        print("📍 Position yourself so your full body is visible")
    elif source.lower() == 'video':
        if not video_path or not os.path.exists(video_path):
            print(f"❌ Error: Video file not found: {video_path}")
            return None
        cap = cv2.VideoCapture(video_path)
        use_duration = False
        # Get video properties
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        print(f"🎥 Starting situp counter with VIDEO: {os.path.basename(video_path)}")
        print(f"📹 Video Info: {total_frames} frames, {fps:.1f} FPS, ~{total_frames/fps:.1f}s duration")
        print("📹 Analyzing video... (Press 'Q' in video window to stop)")
    else:
        print(f"❌ Error: Invalid source '{source}'. Use 'camera' or 'video'")
        return None
    
    if not cap.isOpened():
        print(f"❌ Error: Cannot open {source}!")
        return None
    
    # Countdown for camera mode
    if source.lower() == 'camera':
        print("⏱️  Get ready in 3 seconds...\n")
        for i in range(3, 0, -1):
            print(f"{i}...")
            time.sleep(1)
    
    print("🏁 START!\n")
    
    # Get start time
    start_time = time.time()
    frame_count = 0
    
    # Configure MediaPipe Pose
    # Note: GPU support in MediaPipe is limited on Windows (mainly Linux/Mac)
    # For GPU: Install CUDA toolkit and ensure proper drivers
    pose_config = {
        'min_detection_confidence': 0.5,
        'min_tracking_confidence': 0.5,
        'model_complexity': 1,  # 0=Lite (fast), 1=Full, 2=Heavy (accurate)
        'enable_segmentation': False,
        'smooth_landmarks': True,
        'static_image_mode': False  # False for video/continuous frames (faster)
    }
    
    # Try to use GPU if requested
    # Note: MediaPipe will automatically use GPU if available on supported platforms
    if use_gpu:
        print("⚡ Attempting GPU acceleration (automatic on supported platforms)")
        # MediaPipe automatically uses GPU on Linux/Mac with proper setup
        # On Windows, GPU support is limited regardless of settings
    
    with mp_pose.Pose(**pose_config) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                print("✓ Video processing completed!" if source == 'video' else "❌ Failed to grab frame")
                break
            
            frame_count += 1
            
            # Calculate elapsed time
            elapsed_time = time.time() - start_time
            remaining_time = max(0, duration - elapsed_time)
            
            # Progress indicator for video (every 30 frames)
            if source.lower() == 'video' and frame_count % 30 == 0:
                progress = (frame_count / total_frames) * 100 if total_frames > 0 else 0
                print(f"⏳ Processing... {frame_count}/{total_frames} frames ({progress:.1f}%) - Situps: {situp_counter.counter}", end='\r')
            
            # Stop after duration (only for camera mode)
            if use_duration and elapsed_time > duration:
                break
            
            # Flip frame horizontally for mirror view (only for camera)
            if source.lower() == 'camera':
                frame = cv2.flip(frame, 1)
            
            # Convert to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            
            # Make detection
            results = pose.process(image)
            
            # Convert back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            # Process if landmarks detected
            if results.pose_landmarks:
                # Extract features
                landmarks = results.pose_landmarks.landmark
                features = extract_pose_features(landmarks)
                hip_angle = features[0]
                
                # Update counter
                count, stage = situp_counter.update(hip_angle)
                
                # Draw landmarks
                mp_drawing.draw_landmarks(
                    image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                )
                
                # Display info box
                cv2.rectangle(image, (0,0), (350,150), (0,0,0), -1)
                
                # Display count
                cv2.putText(image, f'SITUPS: {count}', (10,40), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,255,0), 3)
                
                # Display stage
                stage_color = (0,255,255) if stage == "up" else (255,165,0)
                cv2.putText(image, f'STAGE: {stage if stage else "---"}', (10,80), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.8, stage_color, 2)
                
                # Display time (remaining for camera, elapsed for video)
                if use_duration:
                    cv2.putText(image, f'TIME: {int(remaining_time)}s', (10,120), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
                else:
                    cv2.putText(image, f'TIME: {int(elapsed_time)}s', (10,120), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
                
                # Display angle
                cv2.putText(image, f'ANGLE: {int(hip_angle)}°', (10,150), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200,200,200), 2)
            else:
                # No person detected
                cv2.putText(image, 'NO PERSON DETECTED', (50,240), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            
            # Display frame
            window_title = f'Situp Counter ({source.upper()}) - Press Q to Exit'
            cv2.imshow(window_title, image)
            
            # Exit on 'q' key
            if cv2.waitKey(10) & 0xFF == ord('q'):
                print("\n⚠️  Stopped by user")
                break
    
    # Cleanup
    cap.release()
    cv2.destroyAllWindows()
    
    # Clear progress line
    if source.lower() == 'video':
        print()  # New line after progress
    
    # Final results
    actual_duration = time.time() - start_time
    print(f"\n{'='*60}")
    print(f"🎉 ANALYSIS COMPLETE!")
    print(f"{'='*60}")
    print(f"  Source: {source.upper()}")
    if source.lower() == 'video':
        print(f"  Video: {os.path.basename(video_path)}")
        print(f"  Frames Processed: {frame_count}/{total_frames if source.lower() == 'video' else 'N/A'}")
    print(f"  Total Situps: {situp_counter.counter}")
    print(f"  Duration: {actual_duration:.1f} seconds")
    if actual_duration > 0:
        print(f"  Rate: {situp_counter.counter / (actual_duration/60):.1f} situps/minute")
    print(f"{'='*60}\n")
    
    return situp_counter.counter

print("✓ Situp counter function ready (Camera & Video support)!")

✓ Situp counter function ready (Camera & Video support)!


## 8. Choose Your Input Source

You can analyze situps from either:
- **Camera**: Live webcam feed (30 seconds by default)
- **Video**: Pre-recorded video file (analyzes entire video)

Select the option you want below!

## 9. Alternative: Interactive Option Selector

Run the cell below for an interactive menu to choose camera or video.

## 10. Tips for Best Results

### Camera Position (Both Camera & Video):
1. **Side View**: Position camera to the side to see full body profile
2. **Distance**: Step back so entire body is visible (head to feet)
3. **Lighting**: Ensure good lighting for better pose detection
4. **Clear Background**: Minimal clutter helps with detection

### Proper Situp Form:
1. **Starting Position**: Lie flat with knees bent (feet on floor)
2. **Movement**: Sit up bringing chest toward knees
3. **Full Range**: Complete full situps (down to up) for accurate counting
4. **Consistent Form**: Maintain same form throughout

### Using Video Files:
- Supported formats: `.mp4`, `.avi`, `.mov`, `.mkv`
- Video should show full body throughout
- Place video file in a known location (e.g., `D:\Sports-X\Videos\`)
- Update the path in cell 8 or use the interactive selector in cell 9

### Troubleshooting:
- **No person detected**: Adjust position so whole body is in frame
- **Counter not updating**: Ensure full range of motion in situps
- **Inconsistent counting**: Improve visibility and lighting
- **Video file error**: Check file path and ensure file exists

In [24]:
# Interactive option selector
print("="*60)
print("         SITUP COUNTER - INPUT SOURCE SELECTOR")
print("="*60)
print("\nChoose input source:")
print("  1. Camera (Webcam)")
print("  2. Video File")
print("="*60)

choice = 1

if choice == 1:
    print(f"\n🎥 Starting with CAMERA for 30 seconds...\n")
    result = run_situp_counter(source='camera', duration=30, use_gpu=True)
    print(f"\n✅ Final Result: {result} situps counted!")
    
elif choice == 2:
    # Video option
    video_path = r"D:\Sports-X\Test_data\situps\1.mp4"
    
    # Check if video exists
    if os.path.exists(video_path):
        print(f"\n📹 Starting with VIDEO: {video_path}\n")
        result = run_situp_counter(source='video', video_path=video_path, use_gpu=True)
        print(f"\n✅ Final Result: {result} situps counted!")
    else:
        print(f"\n❌ Error: Video file not found!")
        print(f"   Looking for: {video_path}")
        print(f"\n💡 Make sure the video file exists at this location.")
    
else:
    print("❌ Invalid choice! Please enter 1 or 2.")

         SITUP COUNTER - INPUT SOURCE SELECTOR

Choose input source:
  1. Camera (Webcam)
  2. Video File

🎥 Starting with CAMERA for 30 seconds...

⚡ GPU acceleration requested (limited support on Windows)
🎥 Starting situp counter with WEBCAM for 30 seconds...
📍 Position yourself so your full body is visible
⏱️  Get ready in 3 seconds...

3...
🎥 Starting situp counter with WEBCAM for 30 seconds...
📍 Position yourself so your full body is visible
⏱️  Get ready in 3 seconds...

3...
2...
2...
1...
1...
🏁 START!

⚡ Attempting GPU acceleration (automatic on supported platforms)
🏁 START!

⚡ Attempting GPU acceleration (automatic on supported platforms)

🎉 ANALYSIS COMPLETE!
  Source: CAMERA
  Total Situps: 3
  Duration: 30.4 seconds
  Rate: 5.9 situps/minute


✅ Final Result: 3 situps counted!

🎉 ANALYSIS COMPLETE!
  Source: CAMERA
  Total Situps: 3
  Duration: 30.4 seconds
  Rate: 5.9 situps/minute


✅ Final Result: 3 situps counted!


## Troubleshooting Helper

If you're not getting output, run the cell below to check your video file:

In [ ]:
# Quick Video Check
video_path = r"D:\Sports-X\Test_data\situps\1.mp4"

print("🔍 VIDEO FILE DIAGNOSTICS")
print("=" * 60)
print(f"Video Path: {video_path}")
print(f"File Exists: {os.path.exists(video_path)}")

if os.path.exists(video_path):
    print(f"File Size: {os.path.getsize(video_path) / (1024*1024):.2f} MB")
    
    # Try to open video
    test_cap = cv2.VideoCapture(video_path)
    if test_cap.isOpened():
        width = int(test_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(test_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = test_cap.get(cv2.CAP_PROP_FPS)
        frame_count = int(test_cap.get(cv2.CAP_PROP_FRAME_COUNT))
        duration = frame_count / fps if fps > 0 else 0
        
        print(f"✅ Video can be opened!")
        print(f"   Resolution: {width}x{height}")
        print(f"   FPS: {fps:.2f}")
        print(f"   Total Frames: {frame_count}")
        print(f"   Duration: {duration:.2f} seconds")
        
        # Try to read first frame
        ret, frame = test_cap.read()
        if ret:
            print(f"✅ First frame read successfully!")
            print(f"   Frame shape: {frame.shape}")
        else:
            print("❌ Could not read first frame")
        
        test_cap.release()
    else:
        print("❌ Cannot open video file!")
        print("   The file might be corrupted or in an unsupported format")
else:
    print("❌ File does not exist at this path!")
    print("\n💡 Tips:")
    print("   1. Check if the path is correct")
    print("   2. Make sure the file extension is .mp4, .avi, .mov, or .mkv")
    print("   3. Try using the full absolute path")

print("=" * 60)

🔍 VIDEO FILE DIAGNOSTICS
Video Path: D:\Sports-X\Test_data\situps\1.mp4
File Exists: True
File Size: 10.15 MB
✅ Video can be opened!
   Resolution: 1080x1920
   FPS: 30.00
   Total Frames: 927
   Duration: 30.90 seconds
✅ First frame read successfully!
   Frame shape: (1920, 1080, 3)
